In [1]:
pwd

'c:\\Users\\numai\\Documents\\endtoend\\cnn_classifier\\research'

In [2]:
import os
os.chdir('../')

In [3]:
pwd

'c:\\Users\\numai\\Documents\\endtoend\\cnn_classifier'

In [31]:
from dataclasses import dataclass
from cnn_classifier.utils.common import read_yaml,create_directories
from pathlib import Path
from cnn_classifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH


In [32]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [33]:
class Configuration_Manager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
    def get_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params= self.params
        create_directories([config.root_dir])
        PrepareBaseModel = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate = params.LEARNING_RATE,
            params_include_top = params.INCLUDE_TOP,
            params_weights = params.WEIGHTS,
            params_classes = params.CLASSES
        )
        return PrepareBaseModel


In [34]:
import os
import urllib.request as request
import zipfile as ZipFile
import tensorflow as tf

In [ ]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        self.save_model(path = self.config.base_model_path, model = self.model)
    
    @staticmethod
    def prepare_full_model(model,classes,freeze_all,freeze_till, learning_rate):
        if freeze_all:
            for layers in model.layers:
                layers.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layers in model.layers[:-freeze_till]:
                layers.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units = classes, activation = 'softmax')(flatten_in)
        full_model = tf.keras.models.Model(inputs = model.input, output = prediction)
        full_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                            loss = 'categorical_crossentropy',
                            metrics = ['accuracy'])
        full_model.summary()
        return full_model
    def update_base_model(self):
        self.full_model = self.prepare_full_model(model = self.model,classes = self.config.classes, freeze_all=True, freeze_till=None, learning_rate = self.config.params_learning_rate)
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
    def save_model(path,model):
        model.save(path)


In [36]:
try:
    config = Configuration_Manager()
    config = config.get_base_model_config()
    base_model = PrepareBaseModel(config)
    base_model = base_model.get_base_model()
    updated_base_model  = base_model.update_base_model()
except Exception as e:
    raise e
    

[2026-02-23 20:30:02,150: INFO: common: READ_YAML_VERSION=v3 | reading: C:\Users\numai\Documents\endtoend\cnn_classifier\config\config.yaml]
[2026-02-23 20:30:02,153: INFO: common: yaml file loaded successfully: C:\Users\numai\Documents\endtoend\cnn_classifier\config\config.yaml]
[2026-02-23 20:30:02,153: INFO: common: READ_YAML_VERSION=v3 | reading: C:\Users\numai\Documents\endtoend\cnn_classifier\params.yaml]
[2026-02-23 20:30:02,154: INFO: common: yaml file loaded successfully: C:\Users\numai\Documents\endtoend\cnn_classifier\params.yaml]
[2026-02-23 20:30:02,155: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 33s 1us/step


TypeError: PrepareBaseModel.save_model() got multiple values for argument 'path'